## Portal da Transparência

Nesta pequena PoC estarei demonstrando como extrair os dados do Portal da Transparência

In [2]:
import asyncio
import aiohttp
import time
import json
import re
import nest_asyncio
nest_asyncio.apply()

In [11]:
def check_cpf_info(cpf: str, ano, mes):
    base_url='http://www.portaltransparencia.gov.br/api-de-dados/'
    api_key = {'chave-api-dados': 'a85dfaa630b9a8bcc88a4413bc418d36'}

    tab = '...'
    api_methods = \
    f'''auxilio-emergencial-por-cpf-ou-nis?pagina=1&codigoBeneficiario=
    bpc-por-cpf-ou-nis?pagina=1&codigo=
    bolsa-familia-disponivel-por-cpf-ou-nis?pagina=1&anoMesReferencia={ano+mes}&codigo=
    ceaf?pagina=1&cpfSancionado=
    ceis?pagina=1&codigoSancionado=
    cnep?pagina=1&cnpjSancionado=
    cepim?pagina=1&cnpjSancionado=
    safra-codigo-por-cpf-ou-nis?pagina=1&codigo=
    peti-por-cpf-ou-nis?pagina=1&codigo=
    seguro-defeso-codigo?pagina=1&codigo=
    servidores/remuneracao?pagina=1&mesAno={mes+ano}&cpf=
    viagens-por-cpf?pagina=1&cpf='''.split("\n")

    async def do_request(api_method):
        api_url = api_method + cpf
        try:
            async with aiohttp.ClientSession() as session:
                async with session.get(
                        url = base_url + api_url,
                        headers = api_key
                    ) as http_resp:
                    data = await http_resp.json()
                    pretty_output = json.dumps(data, indent=5)
                    pretty_output = tab + re.sub(r'\n', f'\n{tab}', pretty_output)
                    print(api_url, pretty_output, sep = '\n')
        except Exception as e:
            print(api_url, tab + str(e), sep = '\n')

    async def main(methods):
        ret = await asyncio.gather(*[do_request(method) for method in methods])
        print(f'Todas as solicitações foram finalizadas')


    start = time.time()
    asyncio.run(main(api_methods))
    end = time.time()

    print(f'Levou {end - start} segundos para executar {len(api_methods)} requesições em Paralelo.')


In [13]:
#Exemplo
check_cpf_info('44308720826', '2020', '01')

servidores/remuneracao?pagina=1&mesAno=012019&cpf=44308720826
...{
...     "Error API": "Erro ao executar a consulta"
...}
safra-codigo-por-cpf-ou-nis?pagina=1&codigo=44308720826
...[]
peti-por-cpf-ou-nis?pagina=1&codigo=44308720826
...[]
ceaf?pagina=1&cpfSancionado=44308720826
...[]
seguro-defeso-codigo?pagina=1&codigo=44308720826
...[]
cepim?pagina=1&cnpjSancionado=44308720826
...[]
bolsa-familia-disponivel-por-cpf-ou-nis?pagina=1&anoMesReferencia=201901&codigo=44308720826
...[]
cnep?pagina=1&cnpjSancionado=44308720826
...[]
auxilio-emergencial-por-cpf-ou-nis?pagina=1&codigoBeneficiario=44308720826
...[
...     {
...          "id": 40491559,
...          "mesDisponibilizacao": "04/2020",
...          "beneficiario": {
...               "nis": "20793915397",
...               "nome": "SIMONE BATISTA",
...               "multiploCadastro": false,
...               "cpfFormatado": "***.087.208-**"
...          },
...          "responsavelAuxilioEmergencial": {
...               "nis": "

Acima já conseguimos verificar que o CPF fez uso do auxilio emergencial - entre outras informações.